In [0]:
configs = {
  "fs.azure.account.auth.type": "OAuth",
  "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
  "fs.azure.account.oauth2.client.id": "3",
  "fs.azure.account.oauth2.client.secret": '',
  "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/tensnt_id/oauth2/token"
}

dbutils.fs.mount(
  source="abfss://tokyo-olympic-data@olympicdatapro.dfs.core.windows.net", # container@storageacc
  mount_point="/mnt/tokyoolymicdb",
  extra_configs=configs
)
dbutils.fs.refreshMounts()

---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-1836247928682865>, line 9
      1 configs = {
      2   "fs.azure.account.auth.type": "OAuth",
      3   "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
   (...)
      6   "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/0a97d2f3-8726-417e-b95a-9a4a55f8f5db/oauth2/token"
      7 }
----> 9 dbutils.fs.mount(
     10   source="abfss://tokyo-olympic-data@olympicdatapro.dfs.core.windows.net", # container@storageacc
     11   mount_point="/mnt/tokyoolymicdb",
     12   extra_configs=configs
     13 )
     14 dbutils.fs.refreshMounts()

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    360 exc.__context__ = None
    361 exc.__cause__ = N

In [0]:
%fs
ls "/mnt/tokyoolymic"

path,name,size,modificationTime
dbfs:/mnt/tokyoolymic/excel-files/,excel-files/,0,1714921147000
dbfs:/mnt/tokyoolymic/raw-data/,raw-data/,0,1714921174000
dbfs:/mnt/tokyoolymic/transformed-data/,transformed-data/,0,1714921183000


In [0]:
spark

In [0]:
athletes= spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/athlete.csv")
medals= spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/medals.csv")
teams= spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/teams.csv")
coaches= spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/coaches.csv")
entriesgender= spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/tokyoolymic/raw-data/entriesgender.csv")

In [0]:
medals.show()

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType,DoubleType,FloatType,BooleanType,DataType

In [0]:
entriesgender=entriesgender.withColumn("Female",col("Female").cast(IntegerType()))\
    .withColumn("Male",col("Male").cast(IntegerType()))\
        .withColumn("Total",col("Total").cast(IntegerType()))

In [0]:
entriesgender.printSchema()

In [0]:
medals.printSchema()

In [0]:
#COUNTRIES WITH HIGHEST NUMBER OF GOLD MEDALS
top_gold_medals=medals.orderBy("Gold",ascending=False).select("Team_Country","Gold").show()

In [0]:
athletes.write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/athlete")

In [0]:
coaches.write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/coaches")
teams.write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/teams")
medals.write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/medals")
entriesgender.write.mode("overwrite").option("header","true").csv("/mnt/tokyoolymic/transformed-data/entriesgender")